In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from googlesearch import search
import nltk
import requests

topic = input("Enter topic: ")
for j in search(topic, tld="com", num=10, stop=10, pause=2):
    print(j)
    content = requests.get(j)
    soup = BeautifulSoup(content.text, 'lxml')
    p_tags = soup.find_all('p')
    for paragraph in p_tags:
        print(paragraph.text)

KeyboardInterrupt: 